Data Science Capstone Project Notebook

This notebook will be used to complete the final project in the IBM Data Science course

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
!pip install geocoder
import geocoder
!pip install geopy
from geopy.geocoders import Nominatim
!pip install folium
import folium
import requests
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
print("All modules installed and imported")

     |████████████████████████████████| 98 kB 9.8 MB/s  eta 0:00:01
     |████████████████████████████████| 93 kB 3.9 MB/s  eta 0:00:01
All modules installed and imported


In [120]:
# The code was removed by Watson Studio for sharing.

In [121]:
df = pd.read_csv(body)
df.head()

,Postcode,In Use?,Latitude,Longitude,County,District,London zone,Nearest station,Postcode area,Postcode district,Average Income
0,BR1 1AA,Yes,51.401546,0.015415,Greater London,Bromley,5,Bromley South,BR,BR1,63100
1,BR1 1AB,Yes,51.406333,0.015208,Greater London,Bromley,4,Bromley North,BR,BR1,56100
2,BR1 1AD,No,51.400057,0.016715,Greater London,Bromley,5,Bromley South,BR,BR1,63100
3,BR1 1AE,Yes,51.404543,0.014195,Greater London,Bromley,4,Bromley North,BR,BR1,63100
4,BR1 1AF,Yes,51.401392,0.014948,Greater London,Bromley,5,Bromley South,BR,BR1,63100


In [122]:
df.shape

(323306, 11)

In [123]:
df = df[df["In Use?"].str.contains("Yes")]
df.head()

,Postcode,In Use?,Latitude,Longitude,County,District,London zone,Nearest station,Postcode area,Postcode district,Average Income
0,BR1 1AA,Yes,51.401546,0.015415,Greater London,Bromley,5,Bromley South,BR,BR1,63100
1,BR1 1AB,Yes,51.406333,0.015208,Greater London,Bromley,4,Bromley North,BR,BR1,56100
3,BR1 1AE,Yes,51.404543,0.014195,Greater London,Bromley,4,Bromley North,BR,BR1,63100
4,BR1 1AF,Yes,51.401392,0.014948,Greater London,Bromley,5,Bromley South,BR,BR1,63100
5,BR1 1AG,Yes,51.401392,0.014948,Greater London,Bromley,5,Bromley South,BR,BR1,63100


In [124]:
df = df[df["Postcode district"].str.contains("WC|EC")]
df.shape

(6321, 11)

In [125]:
df.drop(columns=["In Use?", "County", "London zone", "Nearest station", "Postcode area"], inplace = True)
df.head()

,Postcode,Latitude,Longitude,District,Postcode district,Average Income
56706,EC1A 1AA,51.525136,-0.111183,Islington,EC1A,53800
56708,EC1A 1AH,51.524565,-0.112042,Islington,EC1A,53800
56712,EC1A 1AZ,51.524503,-0.112088,Islington,EC1A,53800
56714,EC1A 1BB,51.524565,-0.112042,Islington,EC1A,53800
56720,EC1A 1DN,51.524565,-0.112042,Islington,EC1A,53800


In [126]:
district = df['Postcode district']
df.drop(columns = ["Postcode", "Postcode district"], inplace = True)
df.insert(0, 'Postcode District', district)
df.head()

,Postcode District,Latitude,Longitude,District,Average Income
56706,EC1A,51.525136,-0.111183,Islington,53800
56708,EC1A,51.524565,-0.112042,Islington,53800
56712,EC1A,51.524503,-0.112088,Islington,53800
56714,EC1A,51.524565,-0.112042,Islington,53800
56720,EC1A,51.524565,-0.112042,Islington,53800


In [127]:
df.shape

(6321, 5)

In [128]:
df2 = df.groupby(["Postcode District", "District"]).mean()
df2.head()

Latitude  Longitude  Average Income
Postcode District District                                            
EC1A              City of London  51.518124  -0.100295         65400.0
                  Islington       51.524156  -0.110065         53800.0
EC1M              Camden          51.520813  -0.106501         50000.0
                  City of London  51.520806  -0.098285         65400.0
                  Islington       51.521362  -0.102184         53800.0

In [129]:
df2.shape

(90, 3)

In [130]:
df2.reset_index(inplace = True)
print(df2.shape)
df2.head()

(90, 5)


,Postcode District,District,Latitude,Longitude,Average Income
0,EC1A,City of London,51.518124,-0.100295,65400.0
1,EC1A,Islington,51.524156,-0.110065,53800.0
2,EC1M,Camden,51.520813,-0.106501,50000.0
3,EC1M,City of London,51.520806,-0.098285,65400.0
4,EC1M,Islington,51.521362,-0.102184,53800.0


In [131]:
df2.shape

(90, 5)

In [134]:
address = 'City of London, England'

geolocator = Nominatim(user_agent="Ln_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5156177, -0.0919983.


In [137]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=13)


for lat, lng, postcode, district in zip(df2['Latitude'], df2['Longitude'], df2['Postcode District'], df2["District"]):
    label = '{}, {}'.format(postcode, district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [138]:
# The code was removed by Watson Studio for sharing.

Foursquare credentials set and hidden


In [139]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Postcode District", 
                  "District Latitude", 
                  "District Longitude", 
                  "Venue", 
                  "Venue Latitude", 
                  "Venue Longitude", 
                  "Venue Category"]
    
    return(nearby_venues)

In [140]:
London_venues = getNearbyVenues(names=df2["Postcode District"],
                                   latitudes=df2["Latitude"],
                                   longitudes=df2["Longitude"]
                                  )
London_venues.groupby("Postcode District").count()

EC1A
EC1A
EC1M
EC1M
EC1M
EC1N
EC1N
EC1N
EC1P
EC1R
EC1R
EC1V
EC1V
EC1Y
EC1Y
EC2A
EC2A
EC2A
EC2M
EC2M
EC2M
EC2M
EC2N
EC2N
EC2P
EC2P
EC2R
EC2R
EC2V
EC2V
EC2Y
EC2Y
EC3A
EC3A
EC3M
EC3N
EC3N
EC3N
EC3P
EC3P
EC3R
EC3R
EC3V
EC3V
EC4A
EC4A
EC4A
EC4M
EC4M
EC4N
EC4N
EC4P
EC4R
EC4R
EC4V
EC4V
EC4Y
EC4Y
EC4Y
WC1A
WC1A
WC1B
WC1B
WC1E
WC1E
WC1H
WC1H
WC1N
WC1N
WC1R
WC1V
WC1V
WC1V
WC1X
WC1X
WC2A
WC2A
WC2A
WC2A
WC2B
WC2B
WC2B
WC2E
WC2H
WC2H
WC2H
WC2N
WC2N
WC2R
WC2R


,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode District,,,,,,
EC1A,148,148,148,148,148,148
EC1M,192,192,192,192,192,192
EC1N,245,245,245,245,245,245
EC1P,66,66,66,66,66,66
EC1R,179,179,179,179,179,179
EC1V,161,161,161,161,161,161
EC1Y,179,179,179,179,179,179
EC2A,266,266,266,266,266,266
EC2M,315,315,315,315,315,315


In [141]:
London_onehot = pd.get_dummies(London_venues[['Venue Category']], prefix="", prefix_sep="")

London_onehot['Postcode District'] = London_venues['Postcode District'] 

fixed_columns = [London_onehot.columns[-1]] + list(London_onehot.columns[:-1])
London_onehot = London_onehot[fixed_columns]

Ln_grouped = London_onehot.groupby('Postcode District').mean().reset_index()
Ln_grouped.shape

(41, 222)

In [142]:
num_top_venues = 10

for district in Ln_grouped['Postcode District']:
    print("----"+district+"----")
    temp = Ln_grouped[Ln_grouped['Postcode District'] == district].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----EC1A----
                  venue  freq
0                   Pub  0.11
1           Coffee Shop  0.07
2    Italian Restaurant  0.05
3     French Restaurant  0.04
4                   Bar  0.03
5        Sandwich Place  0.03
6                  Café  0.03
7           Pizza Place  0.03
8  Gym / Fitness Center  0.03
9                 Hotel  0.03


----EC1M----
                   venue  freq
0                    Pub  0.08
1            Coffee Shop  0.07
2             Food Truck  0.07
3                  Hotel  0.06
4                   Café  0.05
5      French Restaurant  0.05
6   Gym / Fitness Center  0.05
7     Italian Restaurant  0.04
8  Vietnamese Restaurant  0.03
9               Beer Bar  0.03


----EC1N----
                  venue  freq
0                   Pub  0.11
1           Coffee Shop  0.07
2        Sandwich Place  0.05
3                 Hotel  0.04
4            Food Truck  0.03
5          Cocktail Bar  0.03
6    Italian Restaurant  0.03
7              Wine Bar  0.03
8               

In [143]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [144]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Postcode District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

London_venues_sorted = pd.DataFrame(columns=columns)
London_venues_sorted['Postcode District'] = Ln_grouped['Postcode District']

for ind in np.arange(Ln_grouped.shape[0]):
    London_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Ln_grouped.iloc[ind, :], num_top_venues)

London_venues_sorted

,Postcode District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,EC1A,Pub,Coffee Shop,Italian Restaurant,French Restaurant,Sandwich Place,Bar,Pizza Place,Gym / Fitness Center,Hotel,Café
1,EC1M,Pub,Coffee Shop,Food Truck,Hotel,Gym / Fitness Center,Café,French Restaurant,Italian Restaurant,Wine Bar,Modern European Restaurant
2,EC1N,Pub,Coffee Shop,Sandwich Place,Hotel,Italian Restaurant,Food Truck,Cocktail Bar,Wine Bar,Sushi Restaurant,French Restaurant
3,EC1P,Pub,Coffee Shop,Garden,Pizza Place,Cocktail Bar,Sandwich Place,Vietnamese Restaurant,Indian Restaurant,Food Truck,History Museum
4,EC1R,Pub,Coffee Shop,Café,Bar,Food Truck,Pizza Place,Vietnamese Restaurant,Middle Eastern Restaurant,Gym / Fitness Center,Breakfast Spot
5,EC1V,Coffee Shop,Pub,Italian Restaurant,Café,Bar,Sandwich Place,Vietnamese Restaurant,Cocktail Bar,Gym / Fitness Center,Restaurant
6,EC1Y,Coffee Shop,Gym / Fitness Center,Food Truck,Italian Restaurant,Pub,Café,Bar,Concert Hall,Turkish Restaurant,Sushi Restaurant
7,EC2A,Coffee Shop,Gym / Fitness Center,Food Truck,Hotel,Pub,Italian Restaurant,Pizza Place,Bar,Café,Cocktail Bar
8,EC2M,Pub,Hotel,Coffee Shop,Food Truck,Pizza Place,Gym / Fitness Center,Café,Bar,Cocktail Bar,French Restaurant
9,EC2N,Pub,Coffee Shop,Hotel,Cocktail Bar,Restaurant,Sandwich Place,Indian Restaurant,Gym / Fitness Center,Pizza Place,Garden


In [145]:
kclusters = 5

Ln_grouped_clustering = Ln_grouped.drop('Postcode District', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Ln_grouped_clustering)

kmeans.labels_[0:10] 

array([4, 1, 4, 2, 4, 1, 1, 1, 2, 2], dtype=int32)

In [149]:
London_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Ln_merged = df2

Ln_merged = Ln_merged.join(London_venues_sorted.set_index('Postcode District'), on='Postcode District')

Ln_merged

,Postcode District,District,Latitude,Longitude,Average Income,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,EC1A,City of London,51.518124,-0.100295,65400.000000,4,Pub,Coffee Shop,Italian Restaurant,French Restaurant,Sandwich Place,Bar,Pizza Place,Gym / Fitness Center,Hotel,Café
1,EC1A,Islington,51.524156,-0.110065,53800.000000,4,Pub,Coffee Shop,Italian Restaurant,French Restaurant,Sandwich Place,Bar,Pizza Place,Gym / Fitness Center,Hotel,Café
2,EC1M,Camden,51.520813,-0.106501,50000.000000,1,Pub,Coffee Shop,Food Truck,Hotel,Gym / Fitness Center,Café,French Restaurant,Italian Restaurant,Wine Bar,Modern European Restaurant
3,EC1M,City of London,51.520806,-0.098285,65400.000000,1,Pub,Coffee Shop,Food Truck,Hotel,Gym / Fitness Center,Café,French Restaurant,Italian Restaurant,Wine Bar,Modern European Restaurant
4,EC1M,Islington,51.521362,-0.102184,53800.000000,1,Pub,Coffee Shop,Food Truck,Hotel,Gym / Fitness Center,Café,French Restaurant,Italian Restaurant,Wine Bar,Modern European Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,WC2H,Westminster,51.511539,-0.128543,58849.397590,0,Coffee Shop,Theater,Pub,Bakery,Cocktail Bar,Ice Cream Shop,Sushi Restaurant,Hotel,Steakhouse,Indian Restaurant
86,WC2N,Islington,51.524565,-0.112042,53800.000000,0,Pub,Hotel,Coffee Shop,Garden,Bakery,Theater,Cocktail Bar,Ice Cream Shop,Pizza Place,Burger Joint
87,WC2N,Westminster,51.508958,-0.124997,58939.189189,0,Pub,Hotel,Coffee Shop,Garden,Bakery,Theater,Cocktail Bar,Ice Cream Shop,Pizza Place,Burger Joint
88,WC2R,Islington,51.524565,-0.112042,53800.000000,0,Pub,Coffee Shop,Sandwich Place,Cocktail Bar,Bar,Theater,Restaurant,Hotel,Burger Joint,Bakery


In [165]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(Ln_merged['Latitude'], Ln_merged['Longitude'], Ln_merged['Postcode District'], Ln_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [158]:
cl1 = Ln_merged.loc[Ln_merged['Cluster Labels'] == 0, Ln_merged.columns[[1] + list(range(5, Ln_merged.shape[1]))]]
cl1

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,City of London,0,Coffee Shop,Pub,Italian Restaurant,Sandwich Place,Hotel,Cocktail Bar,Garden,Vietnamese Restaurant,French Restaurant,Pizza Place
29,Islington,0,Coffee Shop,Pub,Italian Restaurant,Sandwich Place,Hotel,Cocktail Bar,Garden,Vietnamese Restaurant,French Restaurant,Pizza Place
59,Camden,0,Pub,Coffee Shop,Hotel,Theater,Café,Italian Restaurant,Pizza Place,Sandwich Place,Garden,History Museum
60,Islington,0,Pub,Coffee Shop,Hotel,Theater,Café,Italian Restaurant,Pizza Place,Sandwich Place,Garden,History Museum
61,Camden,0,Pub,Coffee Shop,Hotel,Sandwich Place,Café,Pizza Place,Cocktail Bar,Bar,Theater,Bookstore
62,Islington,0,Pub,Coffee Shop,Hotel,Sandwich Place,Café,Pizza Place,Cocktail Bar,Bar,Theater,Bookstore
63,Camden,0,Coffee Shop,Pub,Bar,Exhibit,Pizza Place,Italian Restaurant,Cocktail Bar,Japanese Restaurant,Hotel,Sandwich Place
64,Islington,0,Coffee Shop,Pub,Bar,Exhibit,Pizza Place,Italian Restaurant,Cocktail Bar,Japanese Restaurant,Hotel,Sandwich Place
65,Camden,0,Pub,Coffee Shop,Hotel,Café,Sandwich Place,Garden,Bakery,Pizza Place,Park,Italian Restaurant
66,Islington,0,Pub,Coffee Shop,Hotel,Café,Sandwich Place,Garden,Bakery,Pizza Place,Park,Italian Restaurant


In [159]:
cl2 = Ln_merged.loc[Ln_merged['Cluster Labels'] == 1, Ln_merged.columns[[1] + list(range(5, Ln_merged.shape[1]))]]
cl2

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Camden,1,Pub,Coffee Shop,Food Truck,Hotel,Gym / Fitness Center,Café,French Restaurant,Italian Restaurant,Wine Bar,Modern European Restaurant
3,City of London,1,Pub,Coffee Shop,Food Truck,Hotel,Gym / Fitness Center,Café,French Restaurant,Italian Restaurant,Wine Bar,Modern European Restaurant
4,Islington,1,Pub,Coffee Shop,Food Truck,Hotel,Gym / Fitness Center,Café,French Restaurant,Italian Restaurant,Wine Bar,Modern European Restaurant
11,Hackney,1,Coffee Shop,Pub,Italian Restaurant,Café,Bar,Sandwich Place,Vietnamese Restaurant,Cocktail Bar,Gym / Fitness Center,Restaurant
12,Islington,1,Coffee Shop,Pub,Italian Restaurant,Café,Bar,Sandwich Place,Vietnamese Restaurant,Cocktail Bar,Gym / Fitness Center,Restaurant
13,City of London,1,Coffee Shop,Gym / Fitness Center,Food Truck,Italian Restaurant,Pub,Café,Bar,Concert Hall,Turkish Restaurant,Sushi Restaurant
14,Islington,1,Coffee Shop,Gym / Fitness Center,Food Truck,Italian Restaurant,Pub,Café,Bar,Concert Hall,Turkish Restaurant,Sushi Restaurant
15,City of London,1,Coffee Shop,Gym / Fitness Center,Food Truck,Hotel,Pub,Italian Restaurant,Pizza Place,Bar,Café,Cocktail Bar
16,Hackney,1,Coffee Shop,Gym / Fitness Center,Food Truck,Hotel,Pub,Italian Restaurant,Pizza Place,Bar,Café,Cocktail Bar
17,Islington,1,Coffee Shop,Gym / Fitness Center,Food Truck,Hotel,Pub,Italian Restaurant,Pizza Place,Bar,Café,Cocktail Bar


In [160]:
cl3 = Ln_merged.loc[Ln_merged['Cluster Labels'] == 2, Ln_merged.columns[[1] + list(range(5, Ln_merged.shape[1]))]]
cl3

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Islington,2,Pub,Coffee Shop,Garden,Pizza Place,Cocktail Bar,Sandwich Place,Vietnamese Restaurant,Indian Restaurant,Food Truck,History Museum
18,City of London,2,Pub,Hotel,Coffee Shop,Food Truck,Pizza Place,Gym / Fitness Center,Café,Bar,Cocktail Bar,French Restaurant
19,Hackney,2,Pub,Hotel,Coffee Shop,Food Truck,Pizza Place,Gym / Fitness Center,Café,Bar,Cocktail Bar,French Restaurant
20,Islington,2,Pub,Hotel,Coffee Shop,Food Truck,Pizza Place,Gym / Fitness Center,Café,Bar,Cocktail Bar,French Restaurant
21,Tower Hamlets,2,Pub,Hotel,Coffee Shop,Food Truck,Pizza Place,Gym / Fitness Center,Café,Bar,Cocktail Bar,French Restaurant
22,City of London,2,Pub,Coffee Shop,Hotel,Cocktail Bar,Restaurant,Sandwich Place,Indian Restaurant,Gym / Fitness Center,Pizza Place,Garden
23,Islington,2,Pub,Coffee Shop,Hotel,Cocktail Bar,Restaurant,Sandwich Place,Indian Restaurant,Gym / Fitness Center,Pizza Place,Garden
24,City of London,2,Pub,Hotel,Coffee Shop,Sandwich Place,Food Truck,Pizza Place,Gym / Fitness Center,Cocktail Bar,Steakhouse,Vietnamese Restaurant
25,Islington,2,Pub,Hotel,Coffee Shop,Sandwich Place,Food Truck,Pizza Place,Gym / Fitness Center,Cocktail Bar,Steakhouse,Vietnamese Restaurant
26,City of London,2,Coffee Shop,Pub,Italian Restaurant,Hotel,Cocktail Bar,Gym / Fitness Center,French Restaurant,Café,Pizza Place,Sandwich Place


In [161]:
cl4 = Ln_merged.loc[Ln_merged['Cluster Labels'] == 3, Ln_merged.columns[[1] + list(range(5, Ln_merged.shape[1]))]]
cl4

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,City of London,3,Hotel,Gym / Fitness Center,Restaurant,Cocktail Bar,Beer Bar,Hotel Bar,Garden,South American Restaurant,Salad Place,Event Space


In [162]:
cl5 = Ln_merged.loc[Ln_merged['Cluster Labels'] == 4, Ln_merged.columns[[1] + list(range(5, Ln_merged.shape[1]))]]
cl5

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,City of London,4,Pub,Coffee Shop,Italian Restaurant,French Restaurant,Sandwich Place,Bar,Pizza Place,Gym / Fitness Center,Hotel,Café
1,Islington,4,Pub,Coffee Shop,Italian Restaurant,French Restaurant,Sandwich Place,Bar,Pizza Place,Gym / Fitness Center,Hotel,Café
5,Camden,4,Pub,Coffee Shop,Sandwich Place,Hotel,Italian Restaurant,Food Truck,Cocktail Bar,Wine Bar,Sushi Restaurant,French Restaurant
6,City of London,4,Pub,Coffee Shop,Sandwich Place,Hotel,Italian Restaurant,Food Truck,Cocktail Bar,Wine Bar,Sushi Restaurant,French Restaurant
7,Islington,4,Pub,Coffee Shop,Sandwich Place,Hotel,Italian Restaurant,Food Truck,Cocktail Bar,Wine Bar,Sushi Restaurant,French Restaurant
9,Camden,4,Pub,Coffee Shop,Café,Bar,Food Truck,Pizza Place,Vietnamese Restaurant,Middle Eastern Restaurant,Gym / Fitness Center,Breakfast Spot
10,Islington,4,Pub,Coffee Shop,Café,Bar,Food Truck,Pizza Place,Vietnamese Restaurant,Middle Eastern Restaurant,Gym / Fitness Center,Breakfast Spot
44,City of London,4,Pub,Coffee Shop,Sandwich Place,Italian Restaurant,Hotel,Restaurant,French Restaurant,Wine Bar,Burrito Place,Sushi Restaurant
45,Islington,4,Pub,Coffee Shop,Sandwich Place,Italian Restaurant,Hotel,Restaurant,French Restaurant,Wine Bar,Burrito Place,Sushi Restaurant
46,Westminster,4,Pub,Coffee Shop,Sandwich Place,Italian Restaurant,Hotel,Restaurant,French Restaurant,Wine Bar,Burrito Place,Sushi Restaurant
